# L1 Language Models, the Chat Format and Tokens

## Setup
#### Load the API key and relevant Python libaries.
In the course, a script is provided that loads an OpenAI API key for you. However, in my implementation, I'm customizing the code to use the GROQ API instead

1. **Install Necessary Libraries**: First, make sure you have the `python-dotenv` library installed. If you haven't installed it yet, you can install it using pip:

   ```bash
   pip install python-dotenv
2. **Create a `.env` File**: Create a file named `.env` in the same directory as your notebook. This file will store your API key. The `.env` file should follow the format: API_KEY="your_api_key_here"

3. **Load Environment Variables in Notebook**: In your Python notebook, you need to load the environment variables from the `.env` file. You can do this using the dotenv library:
   ```bash
   from dotenv import load_dotenv
   import os
   
   # Load environment variables from .env file
   load_dotenv()
   
   #Access the API key
   #api_key = os.getenv("API_KEY")

4. **Add `.env` to .gitignore**: If you're using version control like Git, make sure to add the `.env` file to your .gitignore so that your API key isn't accidentally exposed or committed to your repository.

That's it! Now your API key is stored securely as an environment variable in your Python notebook.






In [37]:
!pip install groq
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [44]:
import groq
# Checking the Groq version
print(groq.__version__)

0.5.0


In [35]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

# the old-fashioned way!

# Set the API key as an environment variable
os.environ['API_KEY'] = 'your_key_here'
api_key = os.environ.get('API_KEY')

#### helper function
The function used may look familiar if you took the earlier "ChatGPT Prompt Engineering for Developers" Course. 

Throughout the course, they used OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).


**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to produce the desired results.

**Note**: Throughout this and all other lab notebooks documenting my implementation of this course, I will utilize the Groq library. To utilize the Groq API, please register for an account on [this page](https://console.groq.com/keys).  
Below is the modified code we would use instead of the `get_completion` function. In this implementation, I will be using the Mistral AI `llama3-8b-8192` model, but feel free to experiment with different model outputs.


In [4]:
from groq import Groq

client = Groq(
    api_key=api_key,
)

def get_completion(prompt, model="llama3-8b-8192"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1 # 1 seems to give the best results
    )
    return response.choices[0].message.content

## Prompt the model and get a completion

In [5]:
response = get_completion("What is the capital of Palestine?")

In [6]:
print(response)

The capital of Palestine is a matter of debate and controversy. The Palestine Liberation Organization (PLO), which is the national liberation movement fighting for the rights of the Palestinian people, claims that the capital of Palestine is Jerusalem, which is a major city in the Israeli-occupied West Bank.

The Palestinian National Authority, which is the self-governing body established by the Oslo Accords, has claimed East Jerusalem as its intended capital.

However, Israel asserts sovereignty over all of Jerusalem, including East Jerusalem, and considers it to be its undivided capital. The Israeli government has settled thousands of Jewish settlers in East Jerusalem, and has built museums, roads, and other infrastructure.

In recent years, some countries, including Sweden, have recognized East Jerusalem as the capital of the State of Palestine. However, the majority of countries, including the United States, Israel, and most of the European Union member states, do not recognize Jer

## Tokens

In [7]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

The letters in the word "lollipop" are:

L-O-L-L-I-P-O-P

Reversing these letters, I get:

P-O-P-I-L-L-O-L


This is a stark contrat to the gpt-3.5-turbo model used in the course!

In [8]:
response = get_completion("""Take the letters in l-o-l-l-i-p-o-p and reverse them""")

In [9]:
response

'The letters in l-o-l-l-i-p-o-p in reverse order are:\n\np-o-p-i-l-l-o-l'

## Helper function (chat format)
Here's the helper function we'll use in this course.

In [14]:
def get_completion_from_messages(messages, 
                                 model="llama3-8b-8192", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message.content

In [15]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh my! Let me try,

In the garden, oh so bright,
A carrot grew with joy and light!
Its orange hue was a wondrous sight,
This happy carrot, dancing with delight!

Its green top twirled with glee,
As it soaked up the sunshine freely,
No troubles plagued this happy spud,
Just pure joy, with a carrot's good mood!


In [31]:

def get_completion_and_token_count(messages, 
                                   model="llama3-8b-8192", 
                                   temperature=1, 
                                   max_tokens=500):
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message
    
    token_dict = {
        'prompt_tokens': response.usage.prompt_tokens,
        'completion_tokens':response.usage.completion_tokens,
        'total_tokens':response.usage.total_tokens
        }

    return content, token_dict

In [32]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [33]:
print(response.content)

Oh, oh, the joyful sight!
A happy carrot, glowing bright!
Its orange hue, a vibrant flair,
A crispy crunch, beyond compare!

It beams with glee, a veggie delight,
A bubbly burst of sunshine bright!


In [34]:
print(token_dict)

{'prompt_tokens': 39, 'completion_tokens': 49, 'total_tokens': 88}


#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.